In [2]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio

In [3]:
load_dotenv(override=True)

True

In [4]:
instructions1 = "You are a sales agent working for ITKAgentAI, \
A company that develops advanced AI agents capable of transforming the world. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ITKAgentAI, \
A company that develops advanced AI agents capable of transforming the world. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ITKAgentAI, \
A company that develops advanced AI agents capable of transforming the world. \
You write concise, to the point cold emails."

In [5]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model="gpt-4o-mini"
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model="gpt-4o-mini"
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model="gpt-4o-mini"
)

sale_manager = Agent(
        name="Sale Manager",
        instructions="You pick the best cold sales email from the given options. \
        Imagine you are a customer and pick the one you are most likely to respond to. \
        Do not give an explanation; reply with the selected email only.",
        model="gpt-4o-mini"
)

In [6]:
message = "Write a cold sales email to legal tech company"

with trace("Sale Manager cold sale email"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message)
    )

    outputs = [result.final_output for result in results]
    sale_message = "pick the best email from:\n\n" + "\n\nEmail:\n\n".join(outputs)
    result = await Runner.run(sale_manager, sale_message)
    print(f"Best email:\n{result.final_output}")

Best email:
Subject: Let’s Make Legal Tech Less “Boring” and More “Roaring!” 

Hi [Recipient’s Name],

Ever felt like legal tech is kind of like watching paint dry? 🎨😴 Well, hold onto your legal briefs because ITKAgentAI is here to turn your “meh” into a “wow!”

Imagine a world where your legal processes are as smooth as your favorite coffee. ☕ Our advanced AI agents don’t just strut around looking pretty; they dig into data faster than you can say “objection!” They’re designed to streamline workflows, reduce human error, and maybe even let your team take a breather (gasp!). 

What if you could free up hours for your team to focus on strategy instead of sifting through paperwork? 🕵️‍♂️ Whether it’s contract analysis, case management, or document review, we’ve got the AI magic to help you shine even brighter in the legal tech realm.

If you’re intrigued (or just mildly curious), I’d love to chat! How about a brief call next week? I promise I’ll bring the virtual donuts! 🍩

Looking forwa

In [7]:
# Let's just check emails are working for you

@function_tool
def send_email(title: str, body: str):
    """"Send out an email with the given body to all sales prospects 

        Args:
            title: Title of the email
            body: The body of the email"""
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("danganh2710.it@gmail.com")  # Change to your verified sender
    to_email = To("anh.dang.hoang@cbtw.tech")  # Change to your recipient
    content = Content("text/plain", body)
    mail = Mail(from_email, to_email, title, content).get()
    response = sg.client.mail.send.post(request_body=mail)
    print(response.status_code)

send_email

FunctionTool(name='send_email', description='"Send out an email with the given body to all sales prospects ', params_json_schema={'properties': {'title': {'description': 'Title of the email', 'title': 'Title', 'type': 'string'}, 'body': {'description': 'The body of the email', 'title': 'Body', 'type': 'string'}}, 'required': ['title', 'body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001296FE9EC00>, strict_json_schema=True, is_enabled=True)

In [8]:
# Convert all sale agents to tools

description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

tools = [tool1, tool2, tool3, send_email]

In [ ]:
instructions = """
You are a Sales Manager at ITKAgentAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
 
3. Use the send_email tool to send the best email (and only the best email) to the user.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must send ONE email using the send_email tool — never more than one.
"""


sales_manager = Agent(name="Sales Manager Agent", instructions=instructions, tools=tools, model="gpt-4o-mini")

message = "Send a cold sales email addressed to 'John'"

with trace("Sales Manager Agent"):
    result = await Runner.run(sales_manager, message)

202
